Published on February 27, 2024. By Marília Prata, mpwolke

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/understanding-contextual-questions-answers/train.csv
/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm


In [2]:
data = pd.read_csv("/kaggle/input/understanding-contextual-questions-answers/train.csv")

# Shuffle the dataset
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Define the number of samples for each set
train_size = 2000
val_size = 100
test_size = 100

# Ensure the dataset has enough samples
assert len(data) >= train_size + val_size + test_size, "Dataset is too small for the specified sizes."

# Split the dataset
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:train_size + val_size + test_size]

In [3]:
import re

# Install Keras 3 last. 
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


2024-05-17 10:31:45.696475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 10:31:45.696569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 10:31:45.828145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#Gemma

In [4]:
#Create the model using the from_preset method
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' f

In [ ]:
#Example queries
gemma_lm.generate("What is the name of the player with the largest number of votes?", max_length=64)

In [5]:
#Prepare the dataset for fine-tuning
dataset = []
    
for index, row in train_data.iterrows():
    question, answer = row['question'], row['answer']
    template = (f"Question:\n{question}\n\nAnswer:\n{answer}")
    dataset.append(template)

#Enable LoRA

In [6]:
#Enable LoRA for the model and set the LoRA rank to 64.
gemma_lm.backbone.enable_lora(rank=64)

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW 
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [33]:
gemma_lm.fit(dataset, epochs=2, batch_size=1)#Original was Epochs=15

Epoch 1/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1468s 734ms/step - loss: 0.0965 - sparse_categorical_accuracy: 0.7697
Epoch 2/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1523s 761ms/step - loss: 0.0893 - sparse_categorical_accuracy: 0.7834


In [51]:
# Define the full path to save the model in Kaggle environment
saved_model_path = "/kaggle/working/gemma-fine.h5"

# Save the model
gemma_lm.save(saved_model_path)


In [23]:
#4epochs
print(gemma_lm.generate("Question: How many heads of the departments are older than 56 ?", max_length=256))

Question: How many heads of the departments are older than 56 ?

Answer: 

SELECT COUNT(head) FROM table_2043268_1 WHERE age > 56


In [53]:
#6epochs
print(gemma_lm.generate("Question: How many heads of the departments are older than 56 ?", max_length=256))

Question: How many heads of the departments are older than 56 ?

Answer: 

SELECT COUNT(heads_of_departments) FROM table_2306683_10 WHERE age_of_the_department > 56


In [21]:
#4epochs
print(gemma_lm.generate("Question:What are the maximum and minimum budget of the departments?", max_length=64))

Question:What are the maximum and minimum budget of the departments?

Answer:

SELECT MAX(budget), MIN(budget) FROM table_2363283_1


In [36]:
#6epochs
print(gemma_lm.generate("Question: What are the maximum and minimum budget of the departments?", max_length=64))

Question: What are the maximum and minimum budget of the departments?

Answer: 

SELECT MAX(budget), MIN(budget) FROM departments


In [24]:
#4epochs
print(gemma_lm.generate("Question: What are the names of the heads who are born outside the California state?", max_length=256))

Question: What are the names of the heads who are born outside the California state?

Answer: 

SELECT name FROM table_1142110_1 WHERE birth_place = "Outside California"


In [54]:
#6epochs
print(gemma_lm.generate("Question:\nWhat are the names of the heads who are born outside the California state?", max_length=256))

Question:
What are the names of the heads who are born outside the California state?

Answer:
SELECT name FROM heads WHERE birth_location = "outside California"


In [28]:
#4epochs
print(gemma_lm.generate("question : Show the names of sponsors of players whose residence is either Brandon or Birtle", max_length=256))

question : Show the names of sponsors of players whose residence is either Brandon or Birtle.

Answer : SELECT name FROM sponsors WHERE residence IN ('Brandon', 'Birtle')


In [55]:
#6epochs
print(gemma_lm.generate("Question:\nShow the names of sponsors of players whose residence is either Brandon or Birtle", max_length=256))

Question:
Show the names of sponsors of players whose residence is either Brandon or Birtle.

Answer:
SELECT Name FROM sponsors WHERE Residence IN ('Brandon', 'Birtle')


In [29]:
#4epochs
print(gemma_lm.generate("Question : List the total number of horses on farms in ascending order.", max_length=256))

Question : List the total number of horses on farms in ascending order.

Answer : SELECT COUNT(*) FROM Horses


In [56]:
#6epochs
print(gemma_lm.generate("Question:\nList the total number of horses on farms in ascending order.", max_length=256))

Question:
List the total number of horses on farms in ascending order.

Answer:
SELECT COUNT(*) FROM Horses WHERE Owner = "Farm" ORDER BY COUNT(*)


In [31]:
#4epochs
print(gemma_lm.generate("Question : show the most common occupation of players.", max_length=256))

Question : show the most common occupation of players.

Answer : select t1.occupation from player as t1 join player_occupation as t2 on t1.player_id = t2.player_id group by t1.occupation order by count(*) desc limit 1


In [57]:
#6epochs
print(gemma_lm.generate("Question:\nshow the most common occupation of players.", max_length=256))

Question:
show the most common occupation of players.

Answer:
SELECT MAX(occupation) FROM players


In [32]:
#4epochs
print(gemma_lm.generate("Question : What are the hosts of competitions whose theme is not Aliens?", max_length=256))

Question : What are the hosts of competitions whose theme is not Aliens? 

Answer : 

SELECT host FROM table_1166111_1 WHERE competition_theme_s_ NOT LIKE "%aliens%"


In [61]:
#6epochs
print(gemma_lm.generate("Question :\nWhat are the hosts of competitions whose theme is not Aliens?", max_length=256))

Question :
What are the hosts of competitions whose theme is not Aliens?

Answer :
SELECT host FROM table_2562426_1 WHERE competition_theme_s_ = "Not Aliens"


In [71]:
def test_model(test_data,model):
    for question, reference_answer in zip(test_data['question'], test_data['answer']):
        predicted_answer = model.generate(f"Question:\n{question}")
        print("\nCHAT")
        print(predicted_answer)
        print("\n Real Query:")
        print(reference_answer)

In [ ]:
def extract_sql_answer(model_output):
    # Extracts the SQL query part from the model's output
    return model_output.split("Answer:\n")[1].strip()

In [80]:
import difflib

def calculate_similarity(query1, query2):
    return difflib.SequenceMatcher(None, query1, query2).ratio()

In [81]:
def evaluate_model(test_data, model):
    total_similarity = 0
    total_queries = len(test_data['question'])

    for question, reference_answer in zip(test_data['question'], test_data['answer']):
        # Generate the predicted SQL query
        model_output = model.generate(f"Question:\n{question}")
        predicted_answer = extract_sql_answer(model_output)

        # Calculate similarity between the predicted and reference queries
        similarity = calculate_similarity(predicted_answer, reference_answer)
        total_similarity += similarity

    # Calculate average similarity
    avg_similarity = total_similarity / total_queries
    return avg_similarity

In [82]:
model = gemma_lm  # Replace with your actual model

similarity = evaluate_model(test_data, model)
print(f"Average Similarity: {similarity:.2%}")

Average Similarity: 84.18%


In [77]:
def extract_sql_answer(model_output):
    # Extracts the SQL query part from the model's output
    return model_output.split("Answer:\n")[1].strip()

In [78]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML
import difflib

def parse_sql(query):
    parsed = sqlparse.parse(query)
    return parsed[0] if parsed else None

def compare_parsed_sql(parsed_ref, parsed_pred):
    if not parsed_ref or not parsed_pred:
        return 0.0

    # Compare the types of the SQL statements (SELECT, INSERT, etc.)
    if parsed_ref.get_type() != parsed_pred.get_type():
        return 0.0

    ref_tokens = [token.value for token in parsed_ref.flatten() if not token.is_whitespace]
    pred_tokens = [token.value for token in parsed_pred.flatten() if not token.is_whitespace]

    # Use difflib to compare the token sequences
    return difflib.SequenceMatcher(None, ref_tokens, pred_tokens).ratio()

def evaluate_model_with_structure(test_data, model):
    total_similarity = 0
    total_queries = len(test_data['question'])

    for question, reference_answer in zip(test_data['question'], test_data['answer']):
        # Generate the predicted SQL query
        model_output = model.generate(f"Question:\n{question}")
        predicted_answer = extract_sql_answer(model_output)

        # Parse the reference and predicted SQL queries
        parsed_ref = parse_sql(reference_answer)
        parsed_pred = parse_sql(predicted_answer)

        # Calculate structural similarity
        similarity = compare_parsed_sql(parsed_ref, parsed_pred)
        total_similarity += similarity

    # Calculate average similarity
    avg_similarity = total_similarity / total_queries
    return avg_similarity

In [79]:
structural_similarity = evaluate_model_with_structure(test_data, model)
print(f"Average Structural Similarity: {structural_similarity:.2%}")

Average Structural Similarity: 70.67%


In [72]:
test_model(test_data[0:10],gemma_lm)


CHAT
Question:
How many blocks does the player who played 893 minutes have?

Answer:
SELECT blocks FROM table_2572142_1 WHERE minutes_on_pitch = "893"

 Real Query:
SELECT blocks FROM table_24912693_4 WHERE minutes = 893

CHAT
Question:
What is every game on November 21?

Answer:
SELECT games FROM table_name_4 WHERE date = "november 21"

 Real Query:
SELECT game FROM table_27539535_4 WHERE november = 21

CHAT
Question:
In which special stage named Tempo 2 was Sébastien Loeb the leader?

Answer:
SELECT special_stage FROM table_name_90 WHERE leader = "sebastien loeb" AND stage = "tempo 2"

 Real Query:
SELECT stage FROM table_21326205_2 WHERE rally_leader = "Sébastien Loeb" AND name = "Tempo 2"

CHAT
Question:
On what Date was the Record 9–6–4?

Answer:
SELECT date FROM table_name_46 WHERE record = "9–6–4"

 Real Query:
SELECT date FROM table_name_62 WHERE record = "9–6–4"

CHAT
Question:
Which Result has a Venue of home, and a Date of 3 october 1987?

Answer:
SELECT result FROM table_n